# Importing libraries

In [1]:
import os, glob, json, csv, subprocess, sys, re
from git import *
from subprocess import Popen, PIPE
from os import path
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as ply
import plotly
from plotly.offline import plot, init_notebook_mode, iplot

%matplotlib inline

# Defining repository and directories

In [2]:
userhome = os.path.expanduser("~")
repository = userhome + r"/different-diff/dataset/hbase/"
analyze_dir = userhome + r'/different-diff/analyze/analyze_hbase/'

# Loading the earliest affected versions dataset

In [3]:
affected_version = pd.read_csv(analyze_dir + '05_finding_versions/01_affected_version/earliest_version.csv')
cols = ['bug_id','earliest_affected_version','date_release']
aff = affected_version[cols]
aff

,bug_id,earliest_affected_version,date_release
0,HBASE-500,0.1.0,2010-05-18
1,HBASE-501,0.1.0,2010-05-18
2,HBASE-505,0.1.0,2010-05-18
3,HBASE-552,0.1.0,2010-05-18
4,HBASE-554,0.1.0,2010-05-18
5,HBASE-613,0.1.0,2010-05-18
6,HBASE-532,0.1.1,2010-05-18
7,HBASE-681,0.1.2,2010-05-18
8,HBASE-881,0.18.0,2010-05-18
9,HBASE-1155,0.19.0,2010-05-18


# Combining buggyfile data with the earliest versions dataset

In [4]:
algorithms = ['histogram','minimal','myers','patience']

In [5]:
colname = ['bug_id','bugintro_commitID','parent_id','filepath','number_of_buggyline']

In [6]:
#Combining data with earliest versions
for xx, algr in enumerate(algorithms):
    filedata = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/' + algr + '_total_bugline.csv')
    filedata = filedata[colname]

    details = filedata.join(aff.set_index('bug_id')[['earliest_affected_version','date_release']], on='bug_id')
    details.to_csv(analyze_dir + '05_finding_versions/01_affected_version/affected_version_for_' + algr + '.csv')

In [7]:
details

,bug_id,bugintro_commitID,parent_id,filepath,number_of_buggyline,earliest_affected_version,date_release
0,HBASE-17624,e019961150,b2217d1d8f17a8de1a0cf91b74a7170f5ffc1eed,hbase-server/src/main/java/org/apache/hadoop/h...,1,NaN,NaN
1,HBASE-11258,45883c6bd1,2ab2e5c40e6354012f38bd153a77b53335d9b6e8,src/main/java/org/apache/hadoop/hbase/util/Abs...,13,NaN,NaN
2,HBASE-16620,e5d77d095c,44812cf1ed6255649bd0d67b1cfe46940f11fc1a,hbase-server/src/main/java/org/apache/hadoop/h...,5,NaN,NaN
3,HBASE-15650,d815211f5a,6930da781b2142ed8a0e6fc67095453dea38a072,hbase-server/src/main/java/org/apache/hadoop/h...,2,2.0.0-alpha-1RC0,2017-06-07
4,HBASE-3100,af7f5c7d01,516667bc74081244ba5c33304125938172e5776a,src/test/java/org/apache/hadoop/hbase/Abstract...,143,NaN,NaN
5,HBASE-19319,f886716617,5a0881a98b3575d900d483222e2fdfab15159656,hbase-procedure/src/main/java/org/apache/hadoo...,53,NaN,NaN
6,HBASE-15638,95c1dc93fb,cc237c4b78517a6b370c55c8573b14c9edf683de,hbase-server/src/main/java/org/apache/hadoop/h...,4,NaN,NaN
7,HBASE-15638,95c1dc93fb,cc237c4b78517a6b370c55c8573b14c9edf683de,hbase-client/src/main/java/org/apache/hadoop/h...,138,NaN,NaN
8,HBASE-15638,95c1dc93fb,cc237c4b78517a6b370c55c8573b14c9edf683de,hbase-client/src/main/java/org/apache/hadoop/h...,11,NaN,NaN
9,HBASE-18946,010012cbcb,11e82de8a1d782be85a776ec08e8cd6a071185bf,hbase-server/src/test/java/org/apache/hadoop/h...,1,2.0.0-alpha-3RC0,2017-09-14


# Start comparing buglines with the blamed files

# Histogram

In [8]:
filelist_hist = glob.glob(analyze_dir + '04_annotate/01_blame_files/blame_histogram/csv/*')
blamefiles_hist = ';'.join(filelist_hist).split(';')
blamefiles_hist.sort()
print ("Found " + str(len(blamefiles_hist)) + " files") 

Found 2562 files


In [9]:
diff_files_hist = []
for files in glob.iglob(analyze_dir + "03_file-diff/02_buggylines/histogram/*"):
    fn = open(files, 'r')
    fn = fn.read().split('\n')
    
    if "--- /dev/null" not in fn[0]:
        diff_files_hist.append(files)
    else:
        pass
diff_files_hist.sort()
print ("Found " + str(len(diff_files_hist)) + " files")

Found 2562 files


In [10]:
h = 2560
print (blamefiles_hist[h])
print ("\n" + diff_files_hist[h])

/Users/YusufNugroho/different-diff/analyze/analyze_hbase/04_annotate/01_blame_files/blame_histogram/csv/blame_zookeeper.adoc_5fbf80ee5e-38701ea8ec_HBASE-12902_2561.csv

/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/histogram/zookeeper.adoc_5fbf80ee5e-38701ea8ec_HBASE-12902_2251_histogrambuglines_2251.diff


In [11]:
df_algo_files_hist = []
x = 0
while x < len(diff_files_hist):
    sys.stdout.write("\rExtracting file %i" %(x+1))
    sys.stdout.flush()
    df_algo_files_hist.append([diff_files_hist[x]])
    x += 1

df_algo_files_hist

Extracting file 2562

[['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/histogram/AbstractFSWALProvider.java_e019961150-b2217d1d8f_HBASE-17624_1275_histogrambuglines_1275.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/histogram/AbstractHBaseTool.java_45883c6bd1-2ab2e5c40e_HBASE-11258_2494_histogrambuglines_2494.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/histogram/AbstractHBaseTool.java_e5d77d095c-44812cf1ed_HBASE-16620_1359_histogrambuglines_1359.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/histogram/AbstractMemStore.java_d815211f5a-6930da781b_HBASE-15650_1478_histogrambuglines_1478.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/histogram/AbstractMergeTestBase.java_af7f5c7d01-516667bc74_HBASE-3100_682_histogrambuglines_682.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/

In [12]:
#Open csv file from result of git blame for <filename>.java
for num, blame in enumerate(blamefiles_hist):
    sys.stdout.write('\rComparing bugline_files and blame_files: %i' % (num+1) + ' out of %i' % (len(blamefiles_hist)))
    sys.stdout.flush()
    
    fields = ['line_number', 'code', 'date']
    blamefile = pd.read_csv(blame)
    blamefile = blamefile[fields]

    origin_code = []
    for n in range(0, len(blamefile)):
        temp = [blamefile.iloc[n][0], str(blamefile.iloc[n][1]).replace('nan',''), blamefile.iloc[n][2]]
        origin_code.append(temp)
    
    #Open diff_file <filename>.java-HBASE-xxxx_<diffalgorithm>buglines.diff vs blame_<filename>
    for m, df in enumerate(df_algo_files_hist[num]):
        dfname = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), df)
        dfname = dfname.group(0)
        
        diff_file = open(analyze_dir + '03_file-diff/02_buggylines/histogram/' + dfname,'r')
        bugline = diff_file.read().split('\n')

        if bugline[len(bugline)-1] == '':
            for i in range(0,len(bugline)-1):
                bugline[i] = bugline[i][1:]
            del bugline[len(bugline)-1]
        else:
            for i in range(0,len(bugline)):
                bugline[i] = bugline[i][1:]
        
        bug_id = re.compile('(HBASE-[0-9]*)')
        bug_id = bug_id.findall(df)
        
        for vers in range(0,len(aff)):
            if bug_id[0] == aff.iloc[vers][0]:
                version = aff.iloc[vers][1]
                daterelease = aff.iloc[vers][2]
                break;
            else:
                version = 'None'
                daterelease = 'None'
        
        num = 0
        blame_compare = []
        org_code = [el[1] for el in origin_code]
        for y, bugs in enumerate(bugline):
            if bugs in org_code:
                for x in range(num, len(origin_code)):                
                    codex = origin_code[x][1]
                    if bugs == codex:
                        if version != 'None':
                            if origin_code[x][2] <= daterelease:
                                flag = 'bug-introducing change'
                            else:
                                flag = 'incorrect'
                        else:
                            flag = 'unknown'
                        temp = [bug_id[0], origin_code[x][0], bugs, "True", origin_code[x][2], version, daterelease, flag]
                        blame_compare.append(temp)
                        num = origin_code[x][0]
                        break;
            else:
                temp = [bug_id[0], 'None', bugs, "False", 'None', version, daterelease, 'unknown']
                blame_compare.append(temp)
                #break;
        
        fnames = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w+[_]|(?<=histogram\/)(\w+[_])(?<=[_])"), df)
        fnames = fnames.group(0)
        
        #algoname = re.search(("[_]\w+.diff"), df)
        #algoname = algoname.group(0)
        
        blamename = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), blame)
        blamename = blamename.group(0)
        
        with open (analyze_dir + r'04_annotate/02_diff-file_blame-file_comparison/histogram_comparison/diff_' + fnames + "histogram-" + blamename, 'w') as csvfile:
            writers = csv.writer(csvfile, delimiter = ",")
            header = ['bug_id', 'line_number', 'buggy_code', 'does_bugline_from_originfile?', 'date_added_in_originfile', 'earliest_affected_version', 'version_release_date', 'bug_class']
            writers.writerow(header)
            for item in blame_compare:
                writers.writerow(item)
                
print ('\nComparing buglines are complete')

Comparing bugline_files and blame_files: 2562 out of 2562
Comparing buglines are complete


# Minimal

In [13]:
filelist_min = glob.glob(analyze_dir + '04_annotate/01_blame_files/blame_minimal/csv/*')
blamefiles_min = ';'.join(filelist_min).split(';')
blamefiles_min.sort()
print ("Found " + str(len(blamefiles_min)) + " files")

Found 2562 files


In [14]:
diff_files_min = []
for files in glob.iglob(analyze_dir + "03_file-diff/02_buggylines/minimal/*"):
    fn = open(files, 'r')
    fn = fn.read().split('\n')
    
    if "--- /dev/null" not in fn[0]:
        diff_files_min.append(files)
    else:
        pass
diff_files_min.sort()
print ("Found " + str(len(diff_files_min)) + " files")

Found 2562 files


In [15]:
h = 2391
print (blamefiles_min[h])
print ("\n" + diff_files_min[h])

/Users/YusufNugroho/different-diff/analyze/analyze_hbase/04_annotate/01_blame_files/blame_minimal/csv/blame_list.rb_ec8f0ad428-ab97fd3af8_HBASE-3374_2392.csv

/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/minimal/list.rb_ec8f0ad428-ab97fd3af8_HBASE-3374_634_minimalbuglines_634.diff


In [16]:
df_algo_files_min = []
x = 0
while x < len(diff_files_min):
    sys.stdout.write("\rExtracting file %i" %(x+1))
    sys.stdout.flush()
    df_algo_files_min.append([diff_files_min[x]])
    x += 1
    
df_algo_files_min

Extracting file 2562

[['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/minimal/AbstractFSWALProvider.java_e019961150-b2217d1d8f_HBASE-17624_1275_minimalbuglines_1275.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/minimal/AbstractHBaseTool.java_45883c6bd1-2ab2e5c40e_HBASE-11258_2494_minimalbuglines_2494.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/minimal/AbstractHBaseTool.java_e5d77d095c-44812cf1ed_HBASE-16620_1359_minimalbuglines_1359.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/minimal/AbstractMemStore.java_d815211f5a-6930da781b_HBASE-15650_1478_minimalbuglines_1478.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/minimal/AbstractMergeTestBase.java_af7f5c7d01-516667bc74_HBASE-3100_682_minimalbuglines_682.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_fil

In [17]:
#Open csv file from result of git blame for <filename>.java
for num, blame in enumerate(blamefiles_min):
    sys.stdout.write('\rComparing bugline_files and blame_files: %i' % (num+1) + ' out of %i' % (len(blamefiles_min)))
    sys.stdout.flush()
    
    fields = ['line_number', 'code', 'date']
    blamefile = pd.read_csv(blame)
    blamefile = blamefile[fields]

    origin_code = []
    for n in range(0, len(blamefile)):
        temp = [blamefile.iloc[n][0], str(blamefile.iloc[n][1]).replace('nan',''), blamefile.iloc[n][2]]
        origin_code.append(temp)
    
    #Open diff_file <filename>.java-HBASE-xxxx_<diffalgorithm>buglines.diff vs blame_<filename>
    for m, df in enumerate(df_algo_files_min[num]):
        dfname = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), df)
        dfname = dfname.group(0)
        
        diff_file = open(analyze_dir + '03_file-diff/02_buggylines/minimal/' + dfname,'r')
        bugline = diff_file.read().split('\n')

        if bugline[len(bugline)-1] == '':
            for i in range(0,len(bugline)-1):
                bugline[i] = bugline[i][1:]
            del bugline[len(bugline)-1]
        else:
            for i in range(0,len(bugline)):
                bugline[i] = bugline[i][1:]
        
        bug_id = re.compile('(HBASE-[0-9]*)')
        bug_id = bug_id.findall(df)
        
        for vers in range(0,len(aff)):
            if bug_id[0] == aff.iloc[vers][0]:
                version = aff.iloc[vers][1]
                daterelease = aff.iloc[vers][2]
                break;
            else:
                version = 'None'
                daterelease = 'None'
        
        num = 0
        blame_compare = []
        org_code = [el[1] for el in origin_code]
        for y, bugs in enumerate(bugline):
            if bugs in org_code:
                for x in range(num, len(origin_code)):                
                    codex = origin_code[x][1]
                    if bugs == codex:
                        if version != 'None':
                            if origin_code[x][2] <= daterelease:
                                flag = 'bug-introducing change'
                            else:
                                flag = 'incorrect'
                        else:
                            flag = 'unknown'
                        temp = [bug_id[0], origin_code[x][0], bugs, "True", origin_code[x][2], version, daterelease, flag]
                        blame_compare.append(temp)
                        num = origin_code[x][0]
                        break;
            else:
                temp = [bug_id[0], 'None', bugs, "False", 'None', version, daterelease, 'unknown']
                blame_compare.append(temp)
                #break;
        
        fnames = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w+[_]|(?<=minimal\/)(\w+[_])(?<=[_])"), df)
        fnames = fnames.group(0)
        
        #algoname = re.search(("[_]\w+.diff"), df)
        #algoname = algoname.group(0)
        
        blamename = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), blame)
        blamename = blamename.group(0)
        
        with open (analyze_dir + r'04_annotate/02_diff-file_blame-file_comparison/minimal_comparison/diff_' + fnames + "minimal-" + blamename, 'w') as csvfile:
            writers = csv.writer(csvfile, delimiter = ",")
            header = ['bug_id', 'line_number', 'buggy_code', 'does_bugline_from_originfile?', 'date_added_in_originfile', 'earliest_affected_version', 'version_release_date', 'bug_class']
            writers.writerow(header)
            for item in blame_compare:
                writers.writerow(item)
                
print ('\nComparing buglines are complete')

Comparing bugline_files and blame_files: 2562 out of 2562
Comparing buglines are complete


# Myers

In [18]:
filelist_myers = glob.glob(analyze_dir + '04_annotate/01_blame_files/blame_myers/csv/*')
blamefiles_myers = ';'.join(filelist_myers).split(';')
blamefiles_myers.sort()
print ("Found " + str(len(blamefiles_myers)) + " files")

Found 2562 files


In [19]:
diff_files_myers = []
for files in glob.iglob(analyze_dir + "03_file-diff/02_buggylines/myers/*"):
    fn = open(files, 'r')
    fn = fn.read().split('\n')
    
    if "--- /dev/null" not in fn[0]:
        diff_files_myers.append(files)
    else:
        pass
diff_files_myers.sort()
print ("Found " + str(len(diff_files_myers)) + " files")

Found 2562 files


In [20]:
h = 820
print (blamefiles_myers[h])
print ("\n" + diff_files_myers[h])

/Users/YusufNugroho/different-diff/analyze/analyze_hbase/04_annotate/01_blame_files/blame_myers/csv/blame_IOError.java_f346a3d85a-7e6e530387_HBASE-697_821.csv

/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/myers/IOError.java_f346a3d85a-7e6e530387_HBASE-697_313_myersbuglines_313.diff


In [21]:
df_algo_files_myers = []
x = 0
while x < len(diff_files_myers):
    sys.stdout.write("\rExtracting file %i" %(x+1))
    sys.stdout.flush()
    df_algo_files_myers.append([diff_files_myers[x]])
    x += 1
    
df_algo_files_myers

Extracting file 2562

[['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/myers/AbstractFSWALProvider.java_e019961150-b2217d1d8f_HBASE-17624_1275_myersbuglines_1275.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/myers/AbstractHBaseTool.java_45883c6bd1-2ab2e5c40e_HBASE-11258_2494_myersbuglines_2494.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/myers/AbstractHBaseTool.java_e5d77d095c-44812cf1ed_HBASE-16620_1359_myersbuglines_1359.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/myers/AbstractMemStore.java_d815211f5a-6930da781b_HBASE-15650_1478_myersbuglines_1478.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/myers/AbstractMergeTestBase.java_af7f5c7d01-516667bc74_HBASE-3100_682_myersbuglines_682.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines

In [22]:
#Open csv file from result of git blame for <filename>.java
for num, blame in enumerate(blamefiles_myers):
    sys.stdout.write('\rComparing bugline_files and blame_files: %i' % (num+1) + ' out of %i' % (len(blamefiles_myers)))
    sys.stdout.flush()
    
    fields = ['line_number', 'code', 'date']
    blamefile = pd.read_csv(blame)
    blamefile = blamefile[fields]

    origin_code = []
    for n in range(0, len(blamefile)):
        temp = [blamefile.iloc[n][0], str(blamefile.iloc[n][1]).replace('nan',''), blamefile.iloc[n][2]]
        origin_code.append(temp)
    
    #Open diff_file <filename>.java-HBASE-xxxx_<diffalgorithm>buglines.diff vs blame_<filename>
    for m, df in enumerate(df_algo_files_myers[num]):
        dfname = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), df)
        dfname = dfname.group(0)
        
        diff_file = open(analyze_dir + '03_file-diff/02_buggylines/myers/' + dfname,'r')
        bugline = diff_file.read().split('\n')

        if bugline[len(bugline)-1] == '':
            for i in range(0,len(bugline)-1):
                bugline[i] = bugline[i][1:]
            del bugline[len(bugline)-1]
        else:
            for i in range(0,len(bugline)):
                bugline[i] = bugline[i][1:]
        
        bug_id = re.compile('(HBASE-[0-9]*)')
        bug_id = bug_id.findall(df)
        
        for vers in range(0,len(aff)):
            if bug_id[0] == aff.iloc[vers][0]:
                version = aff.iloc[vers][1]
                daterelease = aff.iloc[vers][2]
                break;
            else:
                version = 'None'
                daterelease = 'None'
        
        num = 0
        blame_compare = []
        org_code = [el[1] for el in origin_code]
        for y, bugs in enumerate(bugline):
            if bugs in org_code:
                for x in range(num, len(origin_code)):                
                    codex = origin_code[x][1]
                    if bugs == codex:
                        if version != 'None':
                            if origin_code[x][2] <= daterelease:
                                flag = 'bug-introducing change'
                            else:
                                flag = 'incorrect'
                        else:
                            flag = 'unknown'
                        temp = [bug_id[0], origin_code[x][0], bugs, "True", origin_code[x][2], version, daterelease, flag]
                        blame_compare.append(temp)
                        num = origin_code[x][0]
                        break;
            else:
                temp = [bug_id[0], 'None', bugs, "False", 'None', version, daterelease, 'unknown']
                blame_compare.append(temp)
                #break;
        
        fnames = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w+[_]|(?<=myers\/)(\w+[_])(?<=[_])"), df)
        fnames = fnames.group(0)
        
        #algoname = re.search(("[_]\w+.diff"), df)
        #algoname = algoname.group(0)
        
        blamename = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), blame)
        blamename = blamename.group(0)
        
        with open (analyze_dir + r'04_annotate/02_diff-file_blame-file_comparison/myers_comparison/diff_' + fnames + "myers-" + blamename, 'w') as csvfile:
            writers = csv.writer(csvfile, delimiter = ",")
            header = ['bug_id', 'line_number', 'buggy_code', 'does_bugline_from_originfile?', 'date_added_in_originfile', 'earliest_affected_version', 'version_release_date', 'bug_class']
            writers.writerow(header)
            for item in blame_compare:
                writers.writerow(item)
                
print ('\nComparing buglines are complete')

Comparing bugline_files and blame_files: 2562 out of 2562
Comparing buglines are complete


# Patience

In [23]:
filelist_pat = glob.glob(analyze_dir + '04_annotate/01_blame_files/blame_patience/csv/*')
blamefiles_pat = ';'.join(filelist_pat).split(';')
blamefiles_pat.sort()
print ("Found " + str(len(blamefiles_pat)) + " files")

Found 2562 files


In [24]:
diff_files_pat = []
for files in glob.iglob(analyze_dir + "03_file-diff/02_buggylines/patience/*"):
    fn = open(files, 'r')
    fn = fn.read().split('\n')
    
    if "--- /dev/null" not in fn[0]:
        diff_files_pat.append(files)
    else:
        pass
diff_files_pat.sort()
print ("Found " + str(len(diff_files_pat)) + " files")

Found 2562 files


In [25]:
h = 1034
print (blamefiles_pat[h])
print ("\n" + diff_files_pat[h])

/Users/YusufNugroho/different-diff/analyze/analyze_hbase/04_annotate/01_blame_files/blame_patience/csv/blame_MonitoredRPCHandler.java_95c1dc93fb-cc237c4b78_HBASE-15638_1035.csv

/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/patience/MonitoredRPCHandler.java_95c1dc93fb-cc237c4b78_HBASE-15638_1767_patiencebuglines_1767.diff


In [26]:
df_algo_files_pat = []
x = 0
while x < len(diff_files_pat):
    sys.stdout.write("\rExtracting file %i" %(x+1))
    sys.stdout.flush()
    df_algo_files_pat.append([diff_files_pat[x]])
    x += 1
    
df_algo_files_pat

Extracting file 2562

[['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/patience/AbstractFSWALProvider.java_e019961150-b2217d1d8f_HBASE-17624_1275_patiencebuglines_1275.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/patience/AbstractHBaseTool.java_45883c6bd1-2ab2e5c40e_HBASE-11258_2494_patiencebuglines_2494.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/patience/AbstractHBaseTool.java_e5d77d095c-44812cf1ed_HBASE-16620_1359_patiencebuglines_1359.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/patience/AbstractMemStore.java_d815211f5a-6930da781b_HBASE-15650_1478_patiencebuglines_1478.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/03_file-diff/02_buggylines/patience/AbstractMergeTestBase.java_af7f5c7d01-516667bc74_HBASE-3100_682_patiencebuglines_682.diff'],
 ['/Users/YusufNugroho/different-diff/analyze/analyze_hb

In [27]:
#Open csv file from result of git blame for <filename>.java
for num, blame in enumerate(blamefiles_pat):
    sys.stdout.write('\rComparing bugline_files and blame_files: %i' % (num+1) + ' out of %i' % (len(blamefiles_pat)))
    sys.stdout.flush()
    
    fields = ['line_number', 'code', 'date']
    blamefile = pd.read_csv(blame)
    blamefile = blamefile[fields]

    origin_code = []
    for n in range(0, len(blamefile)):
        temp = [blamefile.iloc[n][0], str(blamefile.iloc[n][1]).replace('nan',''), blamefile.iloc[n][2]]
        origin_code.append(temp)
    
    #Open diff_file <filename>.java-HBASE-xxxx_<diffalgorithm>buglines.diff vs blame_<filename>
    for m, df in enumerate(df_algo_files_pat[num]):
        dfname = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), df)
        dfname = dfname.group(0)
        
        diff_file = open(analyze_dir + '03_file-diff/02_buggylines/patience/' + dfname,'r')
        bugline = diff_file.read().split('\n')

        if bugline[len(bugline)-1] == '':
            for i in range(0,len(bugline)-1):
                bugline[i] = bugline[i][1:]
            del bugline[len(bugline)-1]
        else:
            for i in range(0,len(bugline)):
                bugline[i] = bugline[i][1:]
        
        bug_id = re.compile('(HBASE-[0-9]*)')
        bug_id = bug_id.findall(df)
        
        for vers in range(0,len(aff)):
            if bug_id[0] == aff.iloc[vers][0]:
                version = aff.iloc[vers][1]
                daterelease = aff.iloc[vers][2]
                break;
            else:
                version = 'None'
                daterelease = 'None'
        
        num = 0
        blame_compare = []
        org_code = [el[1] for el in origin_code]
        for y, bugs in enumerate(bugline):
            if bugs in org_code:
                for x in range(num, len(origin_code)):                
                    codex = origin_code[x][1]
                    if bugs == codex:
                        if version != 'None':
                            if origin_code[x][2] <= daterelease:
                                flag = 'bug-introducing change'
                            else:
                                flag = 'incorrect'
                        else:
                            flag = 'unknown'
                        temp = [bug_id[0], origin_code[x][0], bugs, "True", origin_code[x][2], version, daterelease, flag]
                        blame_compare.append(temp)
                        num = origin_code[x][0]
                        break;
            else:
                temp = [bug_id[0], 'None', bugs, "False", 'None', version, daterelease, 'unknown']
                blame_compare.append(temp)
                #break;
        
        fnames = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w+[_]|(?<=patience\/)(\w+[_])(?<=[_])"), df)
        fnames = fnames.group(0)
        
        #algoname = re.search(("[_]\w+.diff"), df)
        #algoname = algoname.group(0)
        
        blamename = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), blame)
        blamename = blamename.group(0)
        
        with open (analyze_dir + r'04_annotate/02_diff-file_blame-file_comparison/patience_comparison/diff_' + fnames + "patience-" + blamename, 'w') as csvfile:
            writers = csv.writer(csvfile, delimiter = ",")
            header = ['bug_id', 'line_number', 'buggy_code', 'does_bugline_from_originfile?', 'date_added_in_originfile', 'earliest_affected_version', 'version_release_date', 'bug_class']
            writers.writerow(header)
            for item in blame_compare:
                writers.writerow(item)
                
print ('\nComparing buglines are complete')

Comparing bugline_files and blame_files: 2562 out of 2562
Comparing buglines are complete
